# Extracción de activaciones de la MLP en la capa 8 de LLaMA 3.2

Este tutorial explica detalladamente el proceso para extraer activaciones de 
la capa **MLP 8** del modelo **LLaMA 3.2**, simulando el modo de 
preentrenamiento autoregresivo y guardando dichas activaciones como un dataset 
en Hugging Face.

---

##  Objetivo general

Capturar las activaciones (vectores de salida) de la **capa MLP 8** de LLaMA 
3.2 para utilizarlas como dataset de entrada en el entrenamiento de un 
**autoencoder**.

---

##  Fundamento teórico

En lugar de generar tokens uno a uno, alimentamos el modelo con secuencias 
completas y extraemos las salidas de la capa MLP correspondiente a todos los 
tokens en una pasada por lote. Esta técnica simula el comportamiento del 
modelo durante su preentrenamiento autoregresivo.

Para una secuencia de tokens $t_0, t_1, \dots, t_n$, se generan internamente 
los logits para $t_1, t_2, \dots, t_{n+1}$, y podemos interceptar estos 
valores con *hooks*.

---

##  Requisitos técnicos

- GPU con al menos **24 GB de VRAM** (ej. RTX 3090 / 4090 / A6000), 
provenientes de la plataforma vast.ai, debidamente configurago con su llave
- Acceso a Hugging Face con token autorizado al modelo y con permiso de write.
- Paquetes instalados: `transformers`, `datasets`, `torch`, `numpy`, 
`huggingface_hub`, entre otros.

---

##  Pasos

### 1. Autenticación y carga del modelo

```python
from huggingface_hub import login
# Importa la función login para autenticarte con Hugging Face.
login("TU_TOKEN_AQUI") 
# Inicia sesión con tu token personal (reemplaza "TU_TOKEN_AQUI").
from transformers import AutoModelForCausalLM, AutoTokenizer
# Importa:
# AutoModelForCausalLM: carga modelos de lenguaje autoregresivo.
# AutoTokenizer: carga el tokenizador correspondiente.
import torch
# Importa PyTorch para manejar tensores y modelos.
model_name = "meta-llama/Llama-3.2-1B"
# Define el nombre del modelo que se va a usar.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
# Carga el tokenizador desde Hugging Face usando autenticación.
tokenizer.pad_token = tokenizer.eos_token
# Establece el token de padding como el token de fin de secuencia.

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=True
)
#Carga el modelo LLaMA 3.2-1B desde Hugging Face con los siguientes argumentos:
# torch_dtype=torch.float16: utiliza media precisión para reducir el uso de 
# memoria y acelerar los cálculos en GPU.
#device_map="auto": distribuye automáticamente las partes del modelo entre los 
# dispositivos disponibles (por ejemplo, GPU si existe).
# use_auth_token=True: permite acceder al modelo si está protegido o es 
# privado, utilizando el token ya cargado.
model.eval()
# Coloca el modelo en modo evaluación, lo que desactiva componentes como 
# Dropout. Esto garantiza que los resultados sean deterministas durante la 
# inferencia.
```

---

### 2. Inserción del hook

```python
mlp_outputs = []
# Inicializa una lista vacía donde se almacenarán las salidas capturadas 
# del módulo MLP de una capa específica del modelo.
def capture_mlp_output(module, input, output):
    mlp_outputs.append(output.detach().cpu())
# Define una función hook que captura la salida del módulo MLP:
# output.detach() evita que se mantenga el grafo computacional.
# .cpu() mueve el tensor a la memoria de CPU para facilitar el análisis y 
# reducir uso de memoria en GPU.
layer_idx = 8
# Selecciona el índice de la capa del modelo de la cual se desea capturar 
# la salida del MLP. En este caso, es la capa 8.
mlp_module = model.model.layers[layer_idx].mlp
# Accede directamente al módulo MLP de la capa especificada.
hook_handle = mlp_module.register_forward_hook(capture_mlp_output)
# Registra un hook de avance (forward_hook) en el MLP de la capa seleccionada. 
# Este hook ejecutará la función capture_mlp_output cada vez que se realice 
# una pasada hacia adelante por esa capa.
```

---

### 3. Descarga y preparación del dataset

```python
from datasets import load_dataset
# Importa la función load_dataset de la librería datasets de Hugging Face, 
# que permite cargar datasets públicos con una sola línea de código.
dataset = load_dataset(
    "oscar",
    "unshuffled_deduplicated_es",
    split="train[:1%]",
    trust_remote_code=True
)
# Carga una porción del dataset OSCAR (una colección masiva de textos web 
# multilingües). Se especifica:
# "oscar": el nombre del dataset.
# "unshuffled_deduplicated_es": la versión en español sin duplicados.
# split="train[:1%]": se selecciona solo el 1% del conjunto de entrenamiento 
# para reducir el tamaño.
# trust_remote_code=True: permite ejecutar código personalizado del dataset si 
# existe.
texts = [item["text"] for item in dataset if item["text"].strip() != ""]
# Extrae todos los textos del dataset asegurándose de filtrar entradas vacías 
# (aquellas donde "text" es una cadena vacía o solo contiene espacios).

import json
# Importa el módulo json para trabajar con archivos en formato JSON.
with open("textos_oscar_1porciento.jsonl", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(json.dumps({"text": text}) + "\n")
# Crea un archivo en formato .jsonl (JSON Lines), donde cada línea contiene un 
# texto independiente en formato JSON:
# open(..., "w"): abre el archivo en modo escritura.
# encoding="utf-8": asegura compatibilidad con caracteres especiales del 
# español.
# json.dumps({"text": text}): convierte cada texto a una estructura JSON.
# f.write(... + "\n"): escribe cada texto como una línea separada en el archivo 
# de salida.
```

---

### 4. Tokenización y procesamiento por lotes

```python
from torch.utils.data import DataLoader
# Importa DataLoader desde PyTorch, que permite procesar los datos en pequeños 
# lotes (batches) de manera eficiente durante el entrenamiento o inferencia.
import os
import numpy as np
# Importa los módulos estándar os para manipulación de archivos y numpy para 
# operaciones numéricas si se requieren más adelante.
batch_size = 4
max_length = 256
output_dir = "activaciones_mlp8"
# Se define:
# batch_size: número de textos que se procesarán simultáneamente.
# max_length: longitud máxima de tokens por texto.
# output_dir: nombre del directorio donde se guardarán los archivos de salida.
os.makedirs(output_dir, exist_ok=True)
# Crea el directorio de salida si aún no existe, evitando errores si ya está 
# presente.
texts = []
with open("textos_oscar_1porciento.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        texts.append(json.loads(line)["text"])
# Lee el archivo .jsonl creado previamente, línea por línea, cargando cada 
# texto desde su formato JSON y agregándolo a la lista texts.

def collate_fn(batch_texts): # # Función de tokenización por lote
    return tokenizer(batch_texts, return_tensors="pt", padding=True, 
    truncation=True, max_length=max_length)
# Define una función personalizada collate_fn para preparar cada batch:
# Convierte una lista de textos en tensores PyTorch (return_tensors="pt").
# Aplica padding para igualar longitud dentro del batch.
# Aplica truncamiento si un texto excede max_length.
loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_fn)
# Crea el DataLoader, que generará lotes de textos tokenizados utilizando 
# la función collate_fn y el tamaño de batch especificado.
```

---

### 5. Extracción de activaciones

```python
with torch.no_grad():
#Inicia un contexto en el que se desactiva el cálculo del gradiente. Esto 
# reduce el consumo de memoria y mejora la velocidad, ya que no es necesario el 
# entrenamiento del modelo.
    for i, batch in enumerate(loader):
    # Itera sobre los lotes del DataLoader, extrayendo el índice i del lote y 
    # su contenido batch.
        batch = {k: v.to(model.device) for k, v in batch.items()}
        # Envía todos los tensores del lote al mismo dispositivo en el que 
        # está el modelo (por ejemplo, la GPU) para asegurar compatibilidad.
        mlp_outputs.clear()
        # Limpia la lista mlp_outputs para evitar que se acumulen las salidas 
        # de lotes anteriores.
        _ = model(**batch)
        # Ejecuta una pasada hacia adelante del modelo con el lote actual. 
        # Gracias al hook, se captura la salida del MLP de la capa 8 en 
        # mlp_outputs.

        for j, tensor in enumerate(mlp_outputs):
            # Itera sobre cada tensor capturado del lote actual, junto con su 
            # índice j.
            np.save(os.path.join(output_dir, f"lote_{i*batch_size + j:06d}.npy")
            , tensor.numpy())
            # Guarda cada tensor como un archivo .npy en el directorio 
            # output_dir. El nombre del archivo incluye el índice global del 
            # texto procesado (i * batch_size + j) con relleno de ceros para 
            # mantener el orden.

        if (i + 1) % 10 == 0:
            print(f" Procesados y guardados {(i+1)*batch_size:,} textos")
        # Cada 10 lotes, imprime un mensaje indicando cuántos textos han sido 
        # procesados y guardados hasta ese momento.
```

---

##  Subida del dataset a Hugging Face Hub

```python
from huggingface_hub import HfApi
# Crea una instancia de HfApi para acceder a los métodos de la API de Hugging Face.

api = HfApi()
api.upload_file(
    path_or_fileobj="ruta/del/archivo.zip",
    path_in_repo="archivo.zip",
    repo_id="naraca/mi-dataset-activaciones-llama3_2",
    repo_type="dataset"
)
# Sube un archivo ZIP al repositorio en Hugging Face. Parámetros:
# path_or_fileobj: ruta local al archivo .zip que se desea subir.
# path_in_repo: nombre que tendrá el archivo dentro del repositorio.
# repo_id: identificador del repositorio en Hugging Face (en formato 
# usuario/repositorio).
# repo_type="dataset": indica que se trata de un repositorio de tipo dataset.
```

---

##  Preguntas frecuentes 

- ¿Por qué se eligió la **capa MLP 8**?
Por tratarse de una capa intermedia donde se suelen observar representaciones 
lingüísticas generalizadas útiles para tareas posteriores como compresión 
(autoencoders).

- ¿Qué representan los vectores?
Las activaciones internas de la MLP para cada token, una codificación 
semántica útil para analizar y comprimir el conocimiento interno del modelo.

- ¿Qué simula este proceso?
El modo de preentrenamiento, donde el modelo es alimentado con secuencias 
completas para predecir el próximo token sin usar generación paso a paso.

---

## Checklist de implementación

- [x] Carga de modelo/tokenizador
- [x] Descarga y preprocesamiento del corpus
- [x] Inserción del hook en la capa MLP 8
- [x] Extracción por lotes y guardado
- [x] Dataset con `.npy` por texto
- [x] Subida al repositorio Hugging Face

---

##  Estructura del dataset generado

```bash
activaciones_mlp8/
├── lote_000000.npy
├── lote_000001.npy
├── ...
```

Cada archivo `.npy` contiene un tensor de forma `(longitud_secuencia, 4096)`.

---

## Enlace al repositorio del dataset (scrip de mas abajo)

https://huggingface.co/datasets/naraca/mi-dataset-activaciones-llama3_2/tree/
main

---

## Recomendaciones finales

- Correr con GPU potente (ideal A100/4090).
- Usar `torch_dtype=torch.float16` para optimizar VRAM.
- Comprimir los archivos `.npy` periódicamente para subirlos como `.zip`.
